# 使用 filter

## 准备

In [13]:
%%time

from llama_index.core import SummaryIndex, Document
from llama_index.core import Settings

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

from milvus import default_server
from pymilvus import connections, utility

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core.llms import ChatMessage


from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

CPU times: user 44 µs, sys: 0 ns, total: 44 µs
Wall time: 47.2 µs


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f3f8fcd5960>, id_func=<function default_id_func at 0x7f4055abedd0>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

# 启动milvus服务器

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

# Stop your milvus server
# default_server.stop()

default_server.listen_port

v2.3.5-lite
CPU times: user 3.13 s, sys: 332 ms, total: 3.46 s
Wall time: 6.77 s


19530

In [4]:
%%time

# 初始化全局 llm

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 123 ms, sys: 0 ns, total: 123 ms
Wall time: 122 ms


In [5]:
%%time

# 初始化全局 embedding 模型

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 405 ms, sys: 12 ms, total: 417 ms
Wall time: 416 ms


In [6]:
%%time

import logging
import sys

# 设置系统日志，便于设置level排查

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 309 µs, sys: 36 µs, total: 345 µs
Wall time: 350 µs


In [7]:
%%time

# load documents
documents = SimpleDirectoryReader("./books1/").load_data()
for doc in documents:
    print(doc.doc_id, doc.metadata['file_name'])

e99d887b-212d-43ba-b414-7b6937ef8e0a 孔乙己.txt
023c18d7-8dfe-43be-9c21-698808d640e7 社戏.txt
CPU times: user 6.55 ms, sys: 4.07 ms, total: 10.6 ms
Wall time: 10.2 ms


In [8]:
%%time

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: bef41d6660754031bbc39dd8b5318a3c
Created new connection using: bef41d6660754031bbc39dd8b5318a3c
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection
Successfully created an index on collection: llamacollection
CPU times: user 756 ms, sys: 29.2 ms, total: 785 ms
Wall time: 14.9 s


## 直接使用llm的对话

In [9]:
%%time

gen = llm.stream_chat([ChatMessage(role="user", content="介绍下孔乙己")])
for response in gen:
    print(response.delta, end="", flush=True)

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生在小说《呐喊》中塑造的一个经典人物形象。他是19世纪末中国封建社会的一个小知识分子，以“孔乙己”这一绰号为人所知。孔乙己生活在一个破败的酒馆里，他穷困潦倒，衣衫褴褛，常被人嘲笑和捉弄，但他依然保持着读书人的自尊和清高。

孔乙己的特点包括：他好喝懒做，喜欢在酒馆里与人闲聊，虽然识字却不愿劳动；他精通《四书五经》，但生活技能匮乏，连基本的谋生手段都没有；他常被人取笑他的“茴”字有四种写法，这反映了当时社会对底层知识分子的鄙视和无知。

孔乙己的故事揭示了封建社会的残酷现实，以及知识阶层的悲惨命运。他的形象成为了中国文学中一个深刻的社会寓言，象征着旧时代知识分子的悲剧性和无力感。




CPU times: user 367 ms, sys: 30.5 ms, total: 397 ms
Wall time: 11.1 s


## 使用嵌入的问答

In [10]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.16

## 使用filter

In [15]:
%%time

filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="file_name", 
            value="孔乙己.txt",
            operator='=='
        ),
    ],
)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5,
    filters = filters
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生笔下的一位经典人物，出现在短篇小说《孔乙己》中。他生活在清朝末年的鲁镇，以咸亨酒店为生活场所，主要负责温酒的职务。孔乙己的形象特征鲜明，穿着破旧的长衫，常被人嘲笑他的穷酸和读书人的身份。他虽然有过写账的经历，但因为贫穷和屡次的失职，最终沦落到偷窃的地步，腿也因此被打断。他的故事反映了那个时代的社会冷漠，底层人民的艰难生活以及读书人（秀才）在社会中的尴尬境地。尽管他试图保持自己的尊严，但终究无法逃脱命运的捉弄，在人们的嘲笑声中走向悲剧的结局，成为了旧中国社会的一个缩影。
CPU times: user 344 ms, sys: 25.3 ms, total: 370 ms
Wall time: 20 s
